In [1]:
from lib_preprocessing import *

### Focus on Swiss Stocks:
- ABB (ABBN)
- Credit Suisse (CSGN)
- Nestle (NESN)
- Novartis (NOVN)

### The next goals are:
- Exploratory data analysis and stylized facts
- clean covariance matrices
- rolling window
- forecast LSTM


#### Stylized facts:
- Absence of linear autocorrelation
- Heavy-tailed returns
- Long memory of volatility
- outliers (threshold)

#### Plots:
- r_t vs time
- C (tao)
- log P>(r) vs r, if the tail is convex, P>(r) is heavy-tailed (1-ECDF(x)=RECDF(x)= P>(x) --> statsmodel)
- q-q plot of r vs Gaussian and exponential distribution
- plot C_|r|(tao) with log-log axes

 Implement a function that takes as input price returns and outputs the power-law exponent, 
 the starting point, the likelihood ratio of power-law vs exponential and power-law vs log-normal, 
 and if at all possible creates a pdf plot.

#### Correlation matrix cleaning

- We have very small N...
- rolling window length T
- clip eigenvalues


#### Strategies

- What exactly? 
- Measuring device


In [3]:
df.head(100)

,ABBN.S/,CSGN.S/,NESN.S/,NOVN.S/
datetime,,,,
2016-03-07 09:31:00,-1.0,1.0,-1.0,-1.0
2016-03-07 09:32:00,-1.0,1.0,-1.0,-1.0
2016-03-07 09:33:00,1.0,-1.0,1.0,-1.0
2016-03-07 09:34:00,-1.0,-1.0,1.0,1.0
2016-03-07 09:35:00,1.0,-1.0,-1.0,-1.0
2016-03-07 09:36:00,-1.0,1.0,-1.0,-1.0
2016-03-07 09:37:00,-1.0,-1.0,1.0,-1.0
2016-03-07 09:38:00,-1.0,-1.0,-1.0,-1.0
2016-03-07 09:39:00,-1.0,-1.0,-1.0,-1.0
